In [41]:
import gradio as gr
import pandas as pd

from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]


custom_css = """
/* 外层容器居中 */
.gradio-container {
    max-width: 800px;  /* 控制页面宽度 */
    margin: 0 auto !important;  /* 水平居中 */
    padding: 20px !important;
}

/* 标题居中 */
#title {
    text-align: center;
    font-size: 24px;
    margin-bottom: 30px !important;
}
#subtitle {
    text-align: left;
    font-size: 16px;
    margin-bottom: 2px !important;
}
"""

alloWords = ["A","2","3","4","5","6","7","8","9","T","J","Q","K","X","D"]
def update_card_library(card_library):
    message = []
    for card, count in sorted(card_library.items()):
        if count > 0:
            message = f"{card}: {count}张"
            message.append(message)
    return "\n". join(message)

def greet(iniCard, bottomCard, place):
    card_counts = {'3': 4, '4': 4, '5': 4, '6': 4, '7': 4, '8': 4, '9': 4, 'T': 4,
              'J': 4, 'Q': 4, 'K': 4, 'A': 4, '2': 4, 'X': 1, 'D': 1}

    if len(iniCard) != 17:
        return gr.Error("Error: The initilize cards must be exactly 17 digits long."), '', gr.Textbox(interactive=False)
    if len(bottomCard) != 3:
        return gr.Error("Error: The bottom cards must be exactly 3 digits long."), '', gr.Textbox(interactive=False)\
        
    # Check whether the words is meaningless
    for card in iniCard:
        if card not in card_counts:
            return gr.Error(f"Error: The card {card} you input is meaningless."), '', gr.Textbox(interactive=False)
        card_counts[card] -= 1
        if card_counts[card] < 0:
            return gr.Error(f"Error: The card {card} you input is not Enought."), '', gr.Textbox(interactive=False)
            
    # Landlord Judgment
    if place == "Landlord":
        for card in bottomCard:
            if card not in card_counts:
                return gr.Error(f"Error: The card {card} you input is Meaningless."), '', gr.Textbox(interactive=False)
            card_counts[card] -= 1
            if card_counts[card] < 0:
                return gr.Error(f"Error: The card {card} you input is not Enought."), '', gr.Textbox(interactive=False)
            
    card_library = card_counts
    
    # cardRemain = update_card_library(card_library)
    iniData = [iniCard, bottomCard, place]
    return card_library, bottomCard, gr.Textbox(interactive=True)  # Enable the move input after submit


def Move(playInput):
    for card in list(playInput):
        if card not in alloWords:
            return gr.Error("Error: The cards you inputed has meaningless word")
        
    moveData = [playInput]        
    return f"Hello! {playInput}"

with gr.Blocks(css=custom_css) as demo: 
    ## title
    gr.Markdown("""<h1 id="title">Landlord Card Calculator</h1>""")
    
    ## Initialize
    gr.Markdown("""<h3 id="subtitle">Initialize Hand Card</h3>""")
    with gr.Column():
        state = gr.State(False)
        iniCard = gr.Textbox(label="Your Cards", max_length=17)
        bottomCard = gr.Textbox(label="Bottom Card", max_length=3)
        place = gr.Radio(choices=["Landlord", "Upper Neighbour", "Lower Neighbour"], label="Place")
        submit = gr.Button("Submit", variant="primary")
        

    ## Recording
    gr.Markdown("""<h3 id="subtitle">Card-Playing Record</h3>""")
    with gr.Row():
        bottomCardDisplay = gr.Textbox(label="Bottom Card", interactive=False)
        winnerRateDisplay = gr.Textbox(label="Winner Rate", interactive=False)
    with gr.Row():
        with gr.Row():
            recording = gr.Textbox(label="Recording", interactive=False, lines=9)
        with gr.Row():
            with gr.Column():
                suggestion = gr.Textbox(label="Auto Suggestion", interactive=False, lines=1.5)
                playInput = gr.Textbox(label="Your Move", interactive=False, lines=1.5, max_length=20)
                moveSubmit = gr.Button("Move Submit", variant="primary")
    
    
   ## Final Result
    gr.Markdown("""<h3 id="subtitle">Final Result</h3>""")
    with gr.Row():
        final_result = gr.Textbox(label="Result")


    submit.click(
        fn=greet,
        inputs=[iniCard, bottomCard, place],
        outputs=[recording, bottomCardDisplay, playInput]
    )

    moveSubmit.click(
        fn=Move,
        inputs=[playInput],
        outputs=suggestion
    )

demo.launch(inbrowser=False)

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
